In [152]:
import pandas as pd
import re
from collections import Counter
import jieba

filename = 'movie_comments.csv'
content = pd.read_csv(filename)
articles = content['comment'].tolist()

def token(string):
    # we will learn the regular expression next course.
    return re.findall('\w+', string)

articles_clean = [''.join(token(str(a)))for a in articles]
                      
with open('article_9k.txt', 'w',encoding='utf-8') as f:
    for a in articles_clean:
        f.write(a + '\n')
                      
def cut(string): 
    return list(jieba.cut(string))
                      
TOKEN = []
for i, line in enumerate((open('article_9k.txt',encoding='utf-8'))):
    if i % 1000 == 0: print(i)
    # replace 10000 with a big number when you do your homework. 
    TOKEN += cut(line)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [121]:

words_count = Counter(TOKEN)

TOKEN = [str(t) for t in TOKEN]
TOKEN_2_GRAM = [''.join(TOKEN[i:i+2]) for i in range(len(TOKEN[:-2]))]
words_count_2 = Counter(TOKEN_2_GRAM)

def prob_1(word): 
    return words_count[word] / len(TOKEN)
def prob_2(word1, word2):
    if word1 + word2 in words_count_2: return words_count_2[word1+word2] / words_count[word2]
    else:
        return 1 / len(TOKEN_2_GRAM)
def get_probability(sentence):
    words = cut(sentence)
    
    sentence_pro = 1
    
    for i, word in enumerate(words[:-1]):
        next_ = words[i+1]
        probability = prob_2(word, next_)
        sentence_pro *= probability
    sentence_pro *= prob_1(words[-1])
    return sentence_pro

In [122]:
eat = """
aim => time who action food_1 cooking food_2 ， how
time => 今天 | 昨天 | 前天
who => 我 | 他 | 小明 | 红红 | 龟龟 | 方方
action => 吃了 | 尝了
food_1 => 西红柿 | 尖椒 | 木耳 | 腐竹 | 青椒 | 黄豆拌饭酱
cooking => 炒 | 凉拌
food_2 => 鸡蛋 | 皮蛋 | 肉丝 | 土豆丝 | 茄子 | 油菜
how => 好吃 | 不好吃 | 你猜好不好吃 | 你尝尝
"""

import random

def generate_grammar(grammar_str: str, target, split='=>'):
    grammar = {}   
    for line in grammar_str.split('\n'):
        if not line: continue
        expression, formula = line.split(split)
        formulas = formula.split('|')
        formulas = [f.split() for f in formulas]
        grammar[expression.strip()] = formulas        
#        print(expression, ':', formulas)
        
    return grammar

choice_a_expr = random.choice

def generate_by_grammar(grammar: dict,target: str):
    if target not in grammar: return target
    expr = choice_a_expr(grammar[target])
#    print(expr)
    return ''.join(generate_by_grammar(grammar, t) for t in expr)

def generate_by_str(grammar_str, split, target):
    grammar = generate_grammar(grammar_str, target, split)
    return generate_by_grammar(grammar, target)



In [217]:
sp={}
for i in range(10):
    a = generate_by_str(eat, split='=>', target='aim')
    sp[i] = a,get_probability(a)
sorted_sp = sorted(list(sp.values()),key=lambda x: x[1]) 
print('{} is more possible with Prb {}'.format(sorted_sp[-1][0],sorted_sp[-1][1]))
for i in range(10):
    print('-'*4 + ' {} with probility {}'.format(sorted_sp[i][0],sorted_sp[i][1]))




前天龟龟吃了木耳炒鸡蛋，你尝尝 is more possible with Prb 2.516645057157267e-50
---- 前天小明吃了腐竹凉拌油菜，你猜好不好吃 with probility 7.071074409935043e-61
---- 前天方方吃了青椒炒油菜，你猜好不好吃 with probility 7.071074409935043e-61
---- 前天红红尝了尖椒炒油菜，你尝尝 with probility 8.095988039689272e-61
---- 今天龟龟尝了西红柿凉拌鸡蛋，好吃 with probility 7.124469474926559e-59
---- 前天小明尝了腐竹炒土豆丝，好吃 with probility 7.124469474926559e-59
---- 今天方方尝了木耳炒鸡蛋，你猜好不好吃 with probility 5.136308092663455e-58
---- 今天龟龟吃了腐竹炒茄子，你尝尝 with probility 5.29618422536699e-57
---- 昨天他吃了木耳炒皮蛋，你猜好不好吃 with probility 2.381129854704894e-56
---- 昨天我尝了尖椒凉拌油菜，不好吃 with probility 4.231763880838978e-53
---- 前天龟龟吃了木耳炒鸡蛋，你尝尝 with probility 2.516645057157267e-50
